# 데이 컨트롤러 파이프라인 테스트

`app/day_controller.py`를 단계별로 검증하는 테스트 노트북입니다.

- DayController 생성/속성 확인
- execute_turn 정상 경로(interact)
- execute_turn fallback 경로(알 수 없는 tool -> action)
- decision_log 누적 확인
- singleton/alias 확인 (`get_day_controller`, `get_controller`, `ScenarioController`)


In [ ]:
# Colab에서 새로 시작할 때만 필요합니다. 로컬 실행 시 주석 유지하세요.
# !git clone https://github.com/DEUS-EX-MACHlNA/demo-repository.git
# %cd demo-repository
# !git checkout dev/jinhyeok
# !pwd


## 0. Setup - 프로젝트 루트 탐색, 환경변수 로드, 로깅 설정


In [ ]:
import sys
import logging
from pathlib import Path
from dotenv import load_dotenv

ROOT = Path.cwd()
while not (ROOT / "app").exists() and ROOT != ROOT.parent:
    ROOT = ROOT.parent

if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

load_dotenv(ROOT / ".env")
logging.basicConfig(level=logging.INFO, format="%(name)s | %(message)s")

print(f"Project root: {ROOT}")


## 1. 시나리오 로드 + 테스트용 WorldState 준비


In [ ]:
from app.loader import ScenarioLoader
from app.models import WorldState, NPCState

loader = ScenarioLoader(ROOT / "scenarios")
scenario_ids = loader.list_scenarios()
assert scenario_ids, "시나리오를 찾지 못했습니다."

preferred = ["culprit_ai", "coraline"]
scenario_id = next((sid for sid in preferred if sid in scenario_ids), scenario_ids[0])
assets = loader.load(scenario_id)

npc_ids = assets.get_all_npc_ids()
assert npc_ids, "NPC 데이터가 비어 있습니다."

test_npcs = npc_ids[: min(3, len(npc_ids))]
primary_npc = test_npcs[0]

world = WorldState(
    turn=2,
    npcs={
        npc_id: NPCState(npc_id=npc_id, trust=3, fear=1, suspicion=2, humanity=10)
        for npc_id in test_npcs
    },
    inventory=assets.get_initial_inventory() or assets.get_all_item_ids()[:1],
    vars={"humanity": 6, "total_suspicion": 2},
)

print(f"scenario_id={scenario_id}")
print(f"npcs={list(world.npcs.keys())}")
print(f"inventory={world.inventory}")


## 2. DayController 생성 + 기본 상태 확인


In [ ]:
from app.day_controller import DayController

controller = DayController()
assert controller.decision_log == [], "초기 decision_log는 비어 있어야 합니다."

print("DayController 생성 완료")
print(f"초기 decision_log 길이: {len(controller.decision_log)}")


## 3. execute_turn 정상 경로 테스트 (mock: interact 선택)


In [ ]:
from unittest.mock import patch

mock_selection = {
    "tool_name": "interact",
    "args": {"target": primary_npc, "interact": "안녕"},
}

def fake_interact(**kwargs):
    return {
        "event_description": [f"{kwargs['target']}와 대화했다."],
        "state_delta": {
            "npc_stats": {
                primary_npc: {
                    "trust": world.npcs[primary_npc].trust + 2,
                    "suspicion": world.npcs[primary_npc].suspicion - 1,
                }
            },
            "vars": {"humanity": world.vars["humanity"] + 1},
            "turn_increment": 1,
        },
    }

with patch("app.tools.call_tool", return_value=mock_selection), \
     patch.dict("app.tools.TOOLS", {"interact": fake_interact}, clear=False):
    result = controller.execute_turn("엄마에게 인사한다", world, assets)

print(result)
assert result.state_delta["npc_stats"][primary_npc]["trust"] == 2
assert result.state_delta["npc_stats"][primary_npc]["suspicion"] == -1
assert result.state_delta["vars"]["humanity"] == 1
assert result.state_delta["turn_increment"] == 1
print("정상 경로 테스트 통과")


## 4. execute_turn fallback 경로 테스트 (mock: 알 수 없는 tool)


In [ ]:
from unittest.mock import Mock, patch

fallback_action = Mock(return_value={
    "event_description": ["fallback action 실행"],
    "state_delta": {"vars": {"total_suspicion": world.vars["total_suspicion"] + 3}},
})

with patch("app.tools.call_tool", return_value={"tool_name": "unknown_tool", "args": {}}), \
     patch.dict("app.tools.TOOLS", {"action": fallback_action}, clear=False):
    fallback_result = controller.execute_turn("낯선 행동을 한다", world, assets)

fallback_action.assert_called_once_with(action="낯선 행동을 한다")
assert fallback_result.state_delta["vars"]["total_suspicion"] == 3
print(fallback_result)
print("fallback 경로 테스트 통과")


## 5. decision_log 누적/구조 확인


In [ ]:
print(f"decision_log 길이: {len(controller.decision_log)}")
for idx, log in enumerate(controller.decision_log, start=1):
    print(f"[{idx}] turn={log['turn']} user_input={log['user_input']} tool={log['tool_selection']['tool_name']}")

assert len(controller.decision_log) >= 2
assert controller.decision_log[-1]["user_input"] == "낯선 행동을 한다"
print("decision_log 확인 완료")


## 6. singleton + alias 확인


In [ ]:
import app.day_controller as day_module

day_module._day_controller_instance = None
c1 = day_module.get_day_controller()
c2 = day_module.get_day_controller()

assert c1 is c2
assert day_module.get_controller() is c1
assert day_module.ScenarioController is day_module.DayController

print("singleton/alias 테스트 통과")


## 7. (선택) 실제 execute_turn 스모크 테스트

아래 셀은 실제 Tool Calling + LLM 경로를 타기 때문에 환경에 따라 실패할 수 있습니다.


In [ ]:
# real_controller = DayController()
# real_result = real_controller.execute_turn("주변을 살펴본다", world, assets)
# print(real_result)
